In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_percentage_error

In [28]:
# Calculate MAPE values
def calculate_mape(actual, predicted):
    return np.mean(np.abs((actual - predicted) / actual)) * 100

In [69]:
# Arima
# Load the data
data = pd.read_csv('C:\\Users\\tejas\\Desktop\\DataGenie\\sample_1 daily.csv', parse_dates=['point_timestamp'], index_col='point_timestamp')

# Set the frequency to daily
data.index = pd.date_range(start=data.index.min(), end=data.index.max(), freq='D')

data.head()
data = data.drop('Unnamed: 0', axis = 1)


train_data = data.iloc[:int(len(data)*0.8)] #First 80 % is train data

In [70]:
# Fit the model
order = (1, 1, 1) # using an autoregressive term of order 1, a differencing term of order 1, and a moving average term of order 1
model = sm.tsa.arima.ARIMA(train_data['point_value'], order=order)
results = model.fit()

# Generate predictions I am using the last 20% of the data as testing data
start_date = '2021-03-01'
end_date = '2021-07-27'
preds = results.predict(start=start_date, end=end_date, dynamic=False)

# Calculate mean absolute percentage error (MAPE)
actuals = data.loc[start_date:end_date, 'point_value']
mape = calculate_mape(actuals, preds)
print('MAPE:', mape)

MAPE: 4.475284932821783
